[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RobinL/uk_address_matcher/blob/main/match_example_data.ipynb)


In [1]:
!pip install --pre uk_address_matcher



zsh:1: command not found: pip


In [2]:
import duckdb
import pandas as pd

from uk_address_matcher.post_linkage.analyse_results import (
    distinguishability_summary,
)
from uk_address_matcher.post_linkage.identify_distinguishing_tokens import (
    improve_predictions_using_distinguishing_tokens,
)
from uk_address_matcher import clean_data_using_precomputed_rel_tok_freq, get_linker
import time

pd.options.display.max_colwidth = 1000

pd.options.display.max_colwidth = 1000

# -----------------------------------------------------------------------------
# Step 1: Load in some example data.  If using your own data, it must be in
# the same format as the example data.
# -----------------------------------------------------------------------------
# Any additional columns should be retained as-is by the cleaning code

p_fhrs = "https://github.com/RobinL/uk_address_matcher/raw/main/example_data/fhrs_addresses_sample.parquet"
p_ch = "https://github.com/RobinL/uk_address_matcher/raw/main/example_data/companies_house_addresess_postcode_overlap.parquet"

con = duckdb.connect(database=":memory:")
con.sql(f"CREATE TABLE df_fhrs AS SELECT * FROM read_parquet('{p_fhrs}')")
con.sql(f"CREATE TABLE df_ch AS SELECT * FROM read_parquet('{p_ch}')")
df_fhrs = con.table("df_fhrs")
df_ch = con.table("df_ch")

# Display length of the dataset
print(f"Length of FHRS dataset: {len(df_fhrs.df()):,.0f}")
print(f"Length of Companies House dataset: {len(df_ch.df()):,.0f}")

display(df_fhrs.limit(5).df())
display(df_ch.limit(5).df())


Length of FHRS dataset: 5,000
Length of Companies House dataset: 21,952


,unique_id,source_dataset,address_concat,postcode
0,1543406,fhrs,1 OAK CHILTON DAY CENTRE UNIT 2 MARTINS ROAD CHILTON INDUSTRIAL ESTATE SUDBURY,CO10 2FT
1,1395196,fhrs,38 STATION ROAD SUDBURY SUFFOLK,CO10 2SS
2,1394874,fhrs,33 SWAN STREET BOXFORD SUDBURY SUFFOLK,CO10 5NZ
3,1649158,fhrs,11A FRIARS STREET SUDBURY SUFFOLK,CO10 2AA
4,1689685,fhrs,13-14 MARKET PLACE LAVENHAM SUDBURY SUFFOLK,CO10 9QZ


,unique_id,source_dataset,address_concat,postcode
0,06911165,companies_house,NORFOLK HOUSE 22 -24 MARKET PLACE SWAFFHAM NORFOLK,PE37 7QH
1,13048420,companies_house,10-11 THURLOW STREET BEDFORD,MK40 1LR
2,12870226,companies_house,69 RUNWELL ROAD WICKFORD,SS11 7HL
3,09146129,companies_house,249 ONGAR ROAD BRENTWOOD,CM15 9DZ
4,12061693,companies_house,C/O CJAS 105 HIGH STREET BRENTWOOD,CM14 4RR


In [3]:
# -----------------------------------------------------------------------------
# Step 2: Clean the data/feature engineering to prepare for matching model
# -----------------------------------------------------------------------------

df_fhrs_clean = clean_data_using_precomputed_rel_tok_freq(df_fhrs, con=con)
df_ch_clean = clean_data_using_precomputed_rel_tok_freq(df_ch, con=con)


In [4]:
linker = get_linker(
    df_addresses_to_match=df_fhrs_clean,
    df_addresses_to_search_within=df_ch_clean,
    con=con,
    include_full_postcode_block=True,
    additional_columns_to_retain=["original_address_concat"],
)

df_predict = linker.inference.predict(
    threshold_match_weight=-50, experimental_optimisation=True
)
df_predict_ddb = df_predict.as_duckdbpyrelation()

Blocking time: 0.02 seconds
Predict time: 0.52 seconds


In [5]:
start_time = time.time()
df_predict_improved = improve_predictions_using_distinguishing_tokens(
    df_predict=df_predict_ddb,
    con=con,
    match_weight_threshold=-20,
)

df_predict_improved.show(max_width=500, max_rows=5)

end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

┌─────────────────────┬────────────────────────┬─────────────┬─────────────┬───────────────────────┬────────────────────────────┬─────────────────────────────────┬─────────────────────────────────┬─────────────────────────────────┬───────────────────┬─────────────────────┬───────────────────────────────────────────────────┬────────────┬───────────────────────────────────────────────────────┬────────────┐
│    match_weight     │   match_probability    │ unique_id_r │ unique_id_l │ match_weight_original │ match_probability_original │ distinguishing_tokens_1_count_1 │ distinguishing_tokens_1_count_2 │ distinguishing_tokens_2_count_1 │ punishment_tokens │   missing_tokens    │             original_address_concat_l             │ postcode_l │               original_address_concat_r               │ postcode_r │
│       double        │         double         │   varchar   │   varchar   │        double         │           double           │            varchar[]            │            varchar[]

In [6]:
dsum_1 = distinguishability_summary(
    df_predict=df_predict_ddb, df_addresses_to_match=df_fhrs_clean, con=con
)
dsum_1.show(max_width=500, max_rows=20)

dsum_2 = distinguishability_summary(
    df_predict=df_predict_improved, df_addresses_to_match=df_fhrs_clean, con=con
)
dsum_2.show(max_width=500, max_rows=20)


┌─────────────────────────────┬───────┬────────────┐
│ distinguishability_category │ count │ percentage │
│           varchar           │ int64 │  varchar   │
├─────────────────────────────┼───────┼────────────┤
│ 01: One match only          │   757 │ 15.14%     │
│ 02: Distinguishability > 10 │   605 │ 12.10%     │
│ 03: Distinguishability > 5  │   218 │ 4.36%      │
│ 04: Distinguishability > 1  │   624 │ 12.48%     │
│ 05: Distinguishability > 0  │   126 │ 2.52%      │
│ 06.: Distinguishability = 0 │  1500 │ 30.00%     │
│ 99: No match                │  1170 │ 23.40%     │
└─────────────────────────────┴───────┴────────────┘

┌─────────────────────────────┬───────┬────────────┐
│ distinguishability_category │ count │ percentage │
│           varchar           │ int64 │  varchar   │
├─────────────────────────────┼───────┼────────────┤
│ 01: One match only          │   756 │ 15.12%     │
│ 02: Distinguishability > 10 │   698 │ 13.96%     │
│ 03: Distinguishability > 5  │   452 │ 9.04%

In [7]:
from uk_address_matcher.post_linkage.analyse_results import distinguishability_by_id

distinguishability_by_id(df_predict=df_predict_improved, df_addresses_to_match=df_fhrs_clean, con=con).df().sample(10)


,unique_id_r,unique_id_l,original_address_concat_l,postcode_l,match_probability,match_weight,distinguishability,distinguishability_category,original_address_concat_r,postcode_r
820,1364311,13364687,THE WHITE HOUSE QUEENS SQUARE ATTLEBOROUGH NORFOLK,NR17 2AF,1.000000,23.502815,11.506891,02: Distinguishability > 10,THE WHITE HOUSE QUEENS SQUARE ATTLEBOROUGH NORFOLK,NR17 2AF
2288,1047291,09984988,18 SOUTH WALK SOUTH WALK BASILDON,SS14 1BZ,0.143796,-2.573931,2.000000,04: Distinguishability > 1,4 SOUTH WALK BASILDON ESSEX,SS14 1BZ
2558,832888,06302759,KILVERSTONE HALL KILVERSTONE THETFORD,IP24 2RL,0.969614,4.995924,0.100000,05: Distinguishability > 0,KILVERSTONE ROAD KILVERSTONE THETFORD,IP24 2RL
834,1632022,11157537,39 BROTON DRIVE HALSTEAD ESSEX,CO9 1HB,1.000000,22.696912,10.906891,02: Distinguishability > 10,39 BROTON DRIVE HALSTEAD ESSEX,CO9 1HB
2577,1007949,10885375,UNIT1 WINDSOR ROAD BEDFORD,MK42 9SU,0.931416,3.763477,0.100000,05: Distinguishability > 0,UNIT 1 WINDSOR ROAD BEDFORD,MK42 9SU
3735,1539906,15647582,38 HIGH STREET WATTON NORFOLK,IP25 6AE,0.005221,-7.573931,0.000000,06.: Distinguishability = 0,46 - 48 HIGH STREET WATTON NORFOLK,IP25 6AE
376,1567558,14354494,CROFT COTTAGE WARLEY STREET GREAT WARLEY BRENTWOOD ESSEX,CM13 3LA,0.917543,3.476069,NaN,01: One match only,2 NURSERY BUNGALOW WARLEY STREET GREAT WARLEY BRENTWOOD,CM13 3LA
3581,406691,14347645,97 BRICKHILL DRIVE BEDFORD,MK41 7QF,0.020562,-5.573931,0.000000,06.: Distinguishability = 0,117 BRICKHILL DRIVE BEDFORD,MK41 7QF
272,300750,14499143,THE MARKSHALL ESTATE COGGESHALL ESSEX,CO6 1TG,0.998876,9.795924,NaN,01: One match only,MARKS HALL ROAD COGGESHALL COLCHESTER ESSEX,CO6 1TG
2294,980674,01141891,GREYWALLS 61 CHAPEL STREET BILLERICAY ESSEX,CM12 9LT,0.135471,-2.673931,4.000000,04: Distinguishability > 1,1 CHAPEL STREET BILLERICAY ESSEX,CM12 9LT


In [8]:


sql = """
SELECT * FROM df_predict_ddb WHERE match_weight > 0
QUALIFY row_number() OVER (PARTITION BY unique_id_l ORDER BY match_weight DESC) = 1
order by random()
limit 3
"""

recs = con.sql(sql).df().to_dict(orient="records")


for rec in recs:
    print("-" * 80)
    print(rec["unique_id_l"], rec["original_address_concat_l"])
    print(rec["unique_id_r"], rec["original_address_concat_r"])
    display(linker.visualisations.waterfall_chart([rec]))


--------------------------------------------------------------------------------
08788400 16 NORTH STREET SUDBURY
1395357 16 NORTH STREET SUDBURY SUFFOLK


alt.LayerChart(...)

--------------------------------------------------------------------------------
08193820 C-0 BLOCKMANAGEMENT UK 5 STOUR VALLEY BUSINESS CENTRE BRUNDON LANE SUDBURY SUFFOLK
1395501 DINKY TREATS THE BRUNDON LANE CAFE AT UNIT 5 STOUR VALLEY BUSINESS CENTRE BRUNDON LANE SUDBURY


alt.LayerChart(...)

--------------------------------------------------------------------------------
14972763 44 CHURCH STREET BOCKING BRAINTREE ESSEX
300672 42 CHURCH STREET BOCKING BRAINTREE ESSEX


alt.LayerChart(...)